In [28]:
import minsearch
import json

import openai
from openai import OpenAI

import os

from dotenv import load_dotenv
load_dotenv('../.env')

from elasticsearch import Elasticsearch

from tqdm.auto import tqdm

c:\Users\richa\OneDrive\Documentos\Cursos\DataTalksClub\llm-zoomcamp\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Simple Search

Se crea una función simple que utiliza TF-IDF para llevar las palabras a un espacio vectorial y luego utilizar la similitud de coseno para realizar busqueda similar segun la consulta dada.

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [7]:
q = 'the course has already started, can I still enroll?'

boost = {'question': 3.0, 'section': 0.5}
filter = {'course': 'data-engineering-zoomcamp'}

results = index.search(
    query=q,
    filter_dict=filter,
    boost_dict=boost,
    num_results=3
)

In [8]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# OPEANAI

Se usa la busqueda simple para armar el contexto de mi pregunta la cual será pasada al promt del LLM

In [9]:
OPENAI_API_KEY: str = os.getenv('API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [27]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'It depends on the rules and policies of the institution offering the course. Some institutions may allow late enrollment, while others may have a strict deadline for registration. It is best to contact the institution or the course instructor directly to inquire about late enrollment options.'

In [17]:
def search(query: str):
    boost = {'question': 3.0, 'section': 1}
    filter = {'course': 'data-engineering-zoomcamp'}

    results = index.search(
        query=query,
        filter_dict=filter,
        boost_dict=boost,
        num_results=5
    )

    return results

In [18]:
def build_prompt(query: str, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
query = 'how do I run kafka?'

def rag(query: str):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
rag(query)

'To run Kafka using Java, in the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nTo solve the "NoBrokersAvailable" error, first confirm if the Kafka broker docker container is working by using "docker ps". Then, in the docker-compose yaml file folder, run "docker-compose up -d" to start all the instances.'

In [22]:
rag('the course has already started, can I still enroll?')

'No, you cannot enroll in the course after it has already started. It is recommended to register before the course starts using the provided link.'

# ElasticSearch

In [25]:
es_client = Elasticsearch('http://localhost:9200')
# Verifica si el cliente python esta conectado a una instancia de elasticsearch
print(es_client.ping())

True


In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:48<00:00, 19.58it/s]


In [30]:
query = 'the course has already started, can I still enroll?'

In [31]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [32]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
query

'the course has already started, can I still enroll?'

In [37]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [38]:
rag(query)

'Based on the information provided, yes, you can still join the course after the start date. You are still eligible to submit the homeworks, but make sure to be aware of the deadlines for turning in the final projects.'